In [1]:
import numpy as np
from parse_lsp_data import *
import tensorflow as tf

/home/shared/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from keras.preprocessing import image
from keras import applications
from keras.models import Model,Input
from keras.utils import np_utils
from keras.models import Sequential
from keras.callbacks import CSVLogger
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout,Reshape
from keras.optimizers import SGD,Adam
import keras.backend as K
from keras.layers import Input, Dense, Dropout, Conv2D, MaxPooling2D, Flatten,Activation, Average,GlobalAveragePooling2D

Using TensorFlow backend.


In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11813043745794109285
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15785944679
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16161114909604377105
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 15868438119
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12591459529331690316
physical_device_desc: "device: 1, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:05.0, compute capability: 6.0"
]


In [4]:
train_list, train_label, val_list, val_label, test_list, test_label = getLSPDataset(0.8, 0.1)

Resizing and packing images and labels to lists.

Start to process training dataset
Done processing the training dataset
Start to process validation dataset
Done processing the validation dataset
Start to process test dataset
Done processing the test dataset


In [5]:
train_X = train_list	# (N, 227, 227, 3)
train_Y = train_label	# (N, 2, 14)
print("train X: ", train_list.shape, "val X: ", val_list.shape, "test X: ", test_list.shape)
print("train Y: ", train_label.shape, "val Y: ", val_label.shape, "test Y: ", test_label.shape)

train X:  (1600, 227, 227, 3) val X:  (200, 227, 227, 3) test X:  (200, 227, 227, 3)
train Y:  (1600, 2, 14) val Y:  (200, 2, 14) test Y:  (200, 2, 14)


In [6]:
def build_model():
    stage1_model = applications.densenet.DenseNet121(input_shape=(227,227,3), 
                                                    include_top=False, weights='imagenet')
    last = stage1_model.output
    x = Flatten()(last)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.7)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.7)(x)
    x = Dense(28, activation='relu')(x)
    preds = Reshape((2,14))(x)
    model = Model(stage1_model.input, preds)
    return model

In [7]:
model = build_model()

In [8]:
def euclidean_distance_loss(y_true, y_pred):
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1))

In [9]:
def loss_accuracy(y_true, y_pred):
    diff = tf.square(y_true - y_pred)
    loss = tf.reduce_sum(diff)
    dist = tf.sqrt(tf.reduce_sum(diff, axis = 1))	# (N, 2, 14) -> (N, 14)
    accuracy = tf.reduce_sum(tf.to_int32(tf.greater(1.0, dist))) / 16/14
    return accuracy

In [10]:
model.compile(optimizer=Adam(), loss=euclidean_distance_loss, metrics=[loss_accuracy])

In [11]:
csv_logger = CSVLogger('densenet121.csv', append=True, separator=';')

In [12]:
with tf.device('/device:GPU:0'):
    # fit the model
    history = model.fit(train_X, train_Y, epochs=250,batch_size=32, validation_data = (val_list,val_label),callbacks=[csv_logger])

Train on 1600 samples, validate on 200 samples
Epoch 1/250
1600/1600 [==============================] - 50s 31ms/step - loss: 173.8159 - loss_accuracy: 6.2500e-04 - val_loss: 3860.0699 - val_loss_accuracy: 0.0000e+00
Epoch 2/250
1600/1600 [==============================] - 19s 12ms/step - loss: 152.9198 - loss_accuracy: 8.0357e-04 - val_loss: 124.6728 - val_loss_accuracy: 0.0000e+00
Epoch 3/250
1600/1600 [==============================] - 19s 12ms/step - loss: 149.8443 - loss_accuracy: 0.0018 - val_loss: 452.7553 - val_loss_accuracy: 0.0000e+00
Epoch 4/250
1600/1600 [==============================] - 19s 12ms/step - loss: 153.9985 - loss_accuracy: 0.0012 - val_loss: 2319.0758 - val_loss_accuracy: 0.0000e+00
Epoch 5/250
1600/1600 [==============================] - 19s 12ms/step - loss: 149.1768 - loss_accuracy: 8.0357e-04 - val_loss: 329.9803 - val_loss_accuracy: 0.0000e+00
Epoch 6/250
1600/1600 [==============================] - 19s 12ms/step - loss: 150.3404 - loss_accuracy: 8.0357e-0

1600/1600 [==============================] - 19s 12ms/step - loss: 131.4912 - loss_accuracy: 0.0013 - val_loss: 414.7083 - val_loss_accuracy: 0.0000e+00
Epoch 50/250
1600/1600 [==============================] - 19s 12ms/step - loss: 131.8976 - loss_accuracy: 0.0018 - val_loss: 328.4418 - val_loss_accuracy: 0.0000e+00
Epoch 51/250
1600/1600 [==============================] - 19s 12ms/step - loss: 129.3113 - loss_accuracy: 0.0015 - val_loss: 311.9544 - val_loss_accuracy: 0.0000e+00
Epoch 52/250
1600/1600 [==============================] - 19s 12ms/step - loss: 132.7362 - loss_accuracy: 8.9286e-04 - val_loss: 308.6358 - val_loss_accuracy: 0.0000e+00
Epoch 53/250
1600/1600 [==============================] - 19s 12ms/step - loss: 129.6877 - loss_accuracy: 0.0012 - val_loss: 310.5994 - val_loss_accuracy: 0.0000e+00
Epoch 54/250
1600/1600 [==============================] - 19s 12ms/step - loss: 129.8369 - loss_accuracy: 0.0017 - val_loss: 325.4409 - val_loss_accuracy: 0.0000e+00
Epoch 55/250


1600/1600 [==============================] - 19s 12ms/step - loss: 128.9602 - loss_accuracy: 8.9286e-04 - val_loss: 299.5431 - val_loss_accuracy: 0.0000e+00
Epoch 99/250
1600/1600 [==============================] - 19s 12ms/step - loss: 127.2424 - loss_accuracy: 0.0022 - val_loss: 301.3618 - val_loss_accuracy: 0.0000e+00
Epoch 100/250
1600/1600 [==============================] - 19s 12ms/step - loss: 126.8094 - loss_accuracy: 0.0013 - val_loss: 298.4189 - val_loss_accuracy: 0.0000e+00
Epoch 101/250
1600/1600 [==============================] - 19s 12ms/step - loss: 128.1506 - loss_accuracy: 0.0013 - val_loss: 300.7707 - val_loss_accuracy: 0.0000e+00
Epoch 102/250
1600/1600 [==============================] - 19s 12ms/step - loss: 129.9991 - loss_accuracy: 0.0014 - val_loss: 293.6915 - val_loss_accuracy: 7.1429e-04
Epoch 103/250
1600/1600 [==============================] - 19s 12ms/step - loss: 128.8102 - loss_accuracy: 0.0015 - val_loss: 312.4857 - val_loss_accuracy: 0.0000e+00
Epoch 104

Epoch 147/250
1600/1600 [==============================] - 19s 12ms/step - loss: 127.7501 - loss_accuracy: 0.0017 - val_loss: 300.7142 - val_loss_accuracy: 7.1429e-04
Epoch 148/250
1600/1600 [==============================] - 19s 12ms/step - loss: 128.4664 - loss_accuracy: 0.0012 - val_loss: 303.2743 - val_loss_accuracy: 0.0000e+00
Epoch 149/250
1600/1600 [==============================] - 19s 12ms/step - loss: 129.8305 - loss_accuracy: 0.0012 - val_loss: 294.7759 - val_loss_accuracy: 0.0000e+00
Epoch 150/250
1600/1600 [==============================] - 19s 12ms/step - loss: 126.9675 - loss_accuracy: 0.0014 - val_loss: 297.4034 - val_loss_accuracy: 0.0000e+00
Epoch 151/250
1600/1600 [==============================] - 19s 12ms/step - loss: 127.3950 - loss_accuracy: 0.0011 - val_loss: 306.5498 - val_loss_accuracy: 7.1429e-04
Epoch 152/250
1600/1600 [==============================] - 19s 12ms/step - loss: 125.4608 - loss_accuracy: 0.0017 - val_loss: 307.1624 - val_loss_accuracy: 0.0000e+0

Epoch 196/250
1600/1600 [==============================] - 19s 12ms/step - loss: 126.0121 - loss_accuracy: 0.0015 - val_loss: 283.8058 - val_loss_accuracy: 0.0000e+00
Epoch 197/250
1600/1600 [==============================] - 19s 12ms/step - loss: 128.1292 - loss_accuracy: 0.0017 - val_loss: 291.6923 - val_loss_accuracy: 0.0000e+00
Epoch 198/250
1600/1600 [==============================] - 19s 12ms/step - loss: 126.7340 - loss_accuracy: 0.0014 - val_loss: 274.4857 - val_loss_accuracy: 0.0000e+00
Epoch 199/250
1600/1600 [==============================] - 19s 12ms/step - loss: 130.4865 - loss_accuracy: 0.0012 - val_loss: 283.3124 - val_loss_accuracy: 0.0000e+00
Epoch 200/250
1600/1600 [==============================] - 19s 12ms/step - loss: 128.6265 - loss_accuracy: 0.0011 - val_loss: 276.2734 - val_loss_accuracy: 0.0000e+00
Epoch 201/250
1600/1600 [==============================] - 19s 12ms/step - loss: 127.5347 - loss_accuracy: 0.0012 - val_loss: 284.3443 - val_loss_accuracy: 0.0000e+0

1600/1600 [==============================] - 19s 12ms/step - loss: 126.6621 - loss_accuracy: 0.0021 - val_loss: 238.3051 - val_loss_accuracy: 0.0000e+00
Epoch 246/250
1600/1600 [==============================] - 19s 12ms/step - loss: 124.3227 - loss_accuracy: 0.0019 - val_loss: 232.6647 - val_loss_accuracy: 7.1429e-04
Epoch 247/250
1600/1600 [==============================] - 19s 12ms/step - loss: 122.9109 - loss_accuracy: 0.0012 - val_loss: 239.7292 - val_loss_accuracy: 0.0000e+00
Epoch 248/250
1600/1600 [==============================] - 19s 12ms/step - loss: 121.9705 - loss_accuracy: 0.0016 - val_loss: 238.2647 - val_loss_accuracy: 0.0000e+00
Epoch 249/250
1600/1600 [==============================] - 19s 12ms/step - loss: 120.8739 - loss_accuracy: 0.0021 - val_loss: 228.9356 - val_loss_accuracy: 0.0014
Epoch 250/250
1600/1600 [==============================] - 19s 12ms/step - loss: 125.6000 - loss_accuracy: 9.8214e-04 - val_loss: 240.7791 - val_loss_accuracy: 7.1429e-04


In [13]:
with tf.device('/device:GPU:0'):
    loss,evaluation_metric_loss = model.evaluate(test_list, test_label)

200/200 [==============================] - 1s 4ms/step


In [14]:
print(loss)

234.18685485839845


In [15]:
with tf.device('/device:GPU:0'):
    pred = model.predict(test_list)

In [16]:
print(pred[0])
print(train_Y[0])

[[49.167175 51.031292 51.449158 61.498085 65.01981  63.63407  49.603592
  46.99698  50.50136  67.64647  71.24256  71.77015  59.253044 59.978153]
 [96.773155 77.621605 58.978848 59.5631   78.08154  96.364174 48.205986
  43.848335 32.583057 32.47602  44.174232 49.316513 28.350435 16.537786]]
[[ 50.20740877  83.82652876  85.02721161 130.65316017 172.67706016
  105.43882018 130.65316017  61.01355448  95.83335733 170.27569445
  142.65998874 173.87774302 141.45930588 151.06476874]
 [206.30842279 161.39537968 115.06402995 115.53679882 160.92261081
  191.65258767  83.38851533  84.80682196  53.60407622  53.60407622
   74.87867558  87.64343521  47.45808084  19.56471723]]


In [17]:
model.save('denseNet_v1.h5')